In [15]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/joaquin/Documents/GitHub/skforecast


In [16]:
from skforecast.drift_detection import RangeDriftDetector
from pprint import pprint
from copy import deepcopy

In [17]:
y = pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D'), name="y")
series_df = pd.DataFrame(
    {
        'series_1': pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D')),
        'series_2': pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D')),
    }
)
series_df_long = series_df.stack().to_frame().rename(columns={0: 'y'}).swaplevel(0,1).sort_index()
exog = pd.Series(np.random.rand(5000), index=y.index, name="temp")
exog_df = pd.DataFrame(index=y.index)
for i in range(5):
    exog_df[f"feature_{i}"] = np.random.rand(5000)
for i in range(5):
    exog_df[f"cat_feature_{i}"] = np.random.choice(["A", "B", "C"], 5000)


series_dict = series_df.to_dict(orient='series')
exog_dict = {'series_1': exog_df, 'series_2': exog_df}
exog_df_long = [
    exog_df.copy().assign(series_id='series_1'),
    exog_df.copy().assign(series_id='series_2')
]
exog_df_long = pd.concat(exog_df_long).set_index('series_id', append=True).reorder_levels(['series_id', None]).sort_index()

# FIT

In [18]:
range_detector = RangeDriftDetector()
range_detector.fit(y)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'y': (np.float64(0.00010909915520840308), np.float64(0.9997863965149076))}
['y']


In [19]:
range_detector = RangeDriftDetector()
range_detector.fit(series_df)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(4.4901005799458105e-05),
              np.float64(0.9997749055410741)),
 'series_2': (np.float64(6.280271749758981e-06), np.float64(0.999779891742373))}
['series_1', 'series_2']


In [20]:
range_detector

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(4.4901005799458105e-05), np.float64(0.9997749055410741)), 'series_2': (np.float64(6.280271749758981e-06), np.float64(0.999779891742373))} 
Exogenous value ranges: None 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: None 

In [21]:
range_detector = RangeDriftDetector()
range_detector.fit(series_dict)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(4.4901005799458105e-05),
              np.float64(0.9997749055410741)),
 'series_2': (np.float64(6.280271749758981e-06), np.float64(0.999779891742373))}
['series_1', 'series_2']


In [22]:
range_detector = RangeDriftDetector()
range_detector.fit(series_df_long)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(4.4901005799458105e-05),
              np.float64(0.9997749055410741)),
 'series_2': (np.float64(6.280271749758981e-06), np.float64(0.999779891742373))}
['series_1', 'series_2']


In [23]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'temp': (np.float64(0.0004645038522779199), np.float64(0.9997853734567731))}
['temp']


In [24]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_df)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'cat_feature_0': {'B', 'A', 'C'},
 'cat_feature_1': {'B', 'A', 'C'},
 'cat_feature_2': {'B', 'A', 'C'},
 'cat_feature_3': {'B', 'A', 'C'},
 'cat_feature_4': {'C', 'B', 'A'},
 'feature_0': (np.float64(0.0002191558626745138),
               np.float64(0.9998784961752988)),
 'feature_1': (np.float64(1.3485237646770898e-05),
               np.float64(0.9999957682451185)),
 'feature_2': (np.float64(0.0002903972354638684),
               np.float64(0.9999165682427562)),
 'feature_3': (np.float64(9.682099526342469e-05),
               np.float64(0.9999938513829746)),
 'feature_4': (np.float64(2.1679103396898824e-05),
               np.float64(0.9997456222238131))}
['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'cat_feature_0', 'cat_feature_1', 'cat_feature_2', 'cat_feature_3', 'cat_feature_4']


In [25]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_dict)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'series_1': {'cat_feature_0': {'B', 'A', 'C'},
              'cat_feature_1': {'B', 'A', 'C'},
              'cat_feature_2': {'B', 'A', 'C'},
              'cat_feature_3': {'B', 'A', 'C'},
              'cat_feature_4': {'C', 'B', 'A'},
              'feature_0': (np.float64(0.0002191558626745138),
                            np.float64(0.9998784961752988)),
              'feature_1': (np.float64(1.3485237646770898e-05),
                            np.float64(0.9999957682451185)),
              'feature_2': (np.float64(0.0002903972354638684),
                            np.float64(0.9999165682427562)),
              'feature_3': (np.float64(9.682099526342469e-05),
                            np.float64(0.9999938513829746)),
              'feature_4': (np.float64(2.1679103396898824e-05),
                            np.float64(0.9997456222238131))},
 'series_2': {'cat_feature_0': {'B', 'A', 'C'},
              'cat_feature_1': {'B', 'A', 'C'},
              'cat_feature_2': {'B', 'A',

In [26]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_df_long)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'series_1': {'cat_feature_0': {'B', 'A', 'C'},
              'cat_feature_1': {'B', 'A', 'C'},
              'cat_feature_2': {'B', 'A', 'C'},
              'cat_feature_3': {'B', 'A', 'C'},
              'cat_feature_4': {'C', 'B', 'A'},
              'feature_0': (np.float64(0.0002191558626745138),
                            np.float64(0.9998784961752988)),
              'feature_1': (np.float64(1.3485237646770898e-05),
                            np.float64(0.9999957682451185)),
              'feature_2': (np.float64(0.0002903972354638684),
                            np.float64(0.9999165682427562)),
              'feature_3': (np.float64(9.682099526342469e-05),
                            np.float64(0.9999938513829746)),
              'feature_4': (np.float64(2.1679103396898824e-05),
                            np.float64(0.9997456222238131))},
 'series_2': {'cat_feature_0': {'B', 'A', 'C'},
              'cat_feature_1': {'B', 'A', 'C'},
              'cat_feature_2': {'B', 'A',

# Predict

In [27]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog)
pprint(range_detector.series_values_range_)
pprint(range_detector.exog_values_range_)


{'y': (np.float64(0.00010909915520840308), np.float64(0.9997863965149076))}
{'temp': (np.float64(0.0004645038522779199), np.float64(0.9997853734567731))}


In [28]:
last_window_valid = y.tail(100).copy()
last_window_invalid = y.tail(100).copy()
last_window_invalid.iloc[5] = 10_000    
range_detector.predict(last_window=last_window_valid)
range_detector.predict(last_window=last_window_invalid)
range_detector


All series and exogenous variables are within the training range.


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'y' has one or more values outside the range seen during training [0.00011,          │
│ 0.99979]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

'y' has one or more values outside the range seen during training [0.00011, 0.99979]. 
'y' has one or more values outside the range seen during training [0.00011, 0.99979]. 



RangeDriftDetector 
Series value ranges: {'y': (np.float64(0.00010909915520840308), np.float64(0.9997863965149076))} 
Exogenous value ranges: {'temp': (np.float64(0.0004645038522779199), np.float64(0.9997853734567731))} 
Fitted series: ['y'] 
Fitted exogenous: ['temp'] 

In [29]:
range_detector.fit(series_df)
last_window_valid_df = series_df.tail(100).copy()
last_window_invalid_df = series_df.tail(100).copy()
last_window_invalid_df.iloc[1, [0, 1]] = 10_000
range_detector.predict(last_window=last_window_valid_df)
range_detector.predict(last_window=last_window_invalid_df)
range_detector

All series and exogenous variables are within the training range.


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1' has one or more values outside the range seen during training [0.00004,   │
│ 0.99977]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2' has one or more values outside the range seen during training [0.00001,   │
│ 0.99978]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

'series_2' has one or more values outside the range seen during training [0.00001, 0.99978]. 
'series_2' has one or more values outside the range seen during training [0.00001, 0.99978]. 



RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(4.4901005799458105e-05), np.float64(0.9997749055410741)), 'series_2': (np.float64(6.280271749758981e-06), np.float64(0.999779891742373))} 
Exogenous value ranges: None 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: None 

In [30]:
range_detector.fit(series_dict)
last_window_valid_dict = {key: val.tail(100).copy() for key, val in series_dict.items()}
last_window_invalid_dict = {key: val.tail(100).copy() for key, val in series_dict.items()}
for key in last_window_invalid_dict.keys():
    last_window_invalid_dict[key].iloc[1] = 10_000
range_detector.predict(last_window=last_window_valid_dict)
range_detector.predict(last_window=last_window_invalid_dict)
range_detector

All series and exogenous variables are within the training range.
'series_2' has one or more values outside the range seen during training [0.00001, 0.99978]. 
'series_2' has one or more values outside the range seen during training [0.00001, 0.99978]. 



RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(4.4901005799458105e-05), np.float64(0.9997749055410741)), 'series_2': (np.float64(6.280271749758981e-06), np.float64(0.999779891742373))} 
Exogenous value ranges: None 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: None 

In [31]:
range_detector.fit(series_df_long)
last_window_valid_df_long= series_df_long.groupby(level=0).tail(100).copy()
last_window_invalid_df_long = series_df_long.groupby(level=0).tail(100).copy()
for key in last_window_invalid_df_long.index.get_level_values(0).unique():
    last_window_invalid_df_long.loc[key, pd.Timestamp("2013-06-02")] = 10_000
range_detector.predict(last_window=last_window_valid_df_long)
range_detector.predict(last_window=last_window_invalid_df_long)

All series and exogenous variables are within the training range.


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1' has one or more values outside the range seen during training [0.00004,   │
│ 0.99977]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2' has one or more values outside the range seen during training [0.00001,   │
│ 0.99978]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

'series_2' has one or more values outside the range seen during training [0.00001, 0.99978]. 
'series_2' has one or more values outside the range seen during training [0.00001, 0.99978]. 



(True, ['series_1', 'series_2'], [])

In [32]:
range_detector.fit(series_df, exog=exog)
print(range_detector)
exog_valid = exog.tail(100).copy()
exog_invalid = exog.tail(100).copy()
exog_invalid.iloc[5] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(4.4901005799458105e-05), np.float64(0.9997749055410741)), 'series_2': (np.float64(6.280271749758981e-06), np.float64(0.999779891742373))} 
Exogenous value ranges: {'temp': (np.float64(0.0004645038522779199), np.float64(0.9997853734567731))} 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: ['temp'] 

All series and exogenous variables are within the training range.


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'temp' has one or more values outside the range seen during training [0.00046,       │
│ 0.99979]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

'temp': 'temp' has one or more values outside the range seen during training [0.00046, 0.99979]. 
'temp': 'temp' has one or more values outside the range seen during training [0.00046, 0.99979]. 



(True, [], ['temp'])

In [33]:
range_detector = RangeDriftDetector()
range_detector.fit(series=y, exog=exog_df)
range_detector
exog_valid = exog_df.tail(100).copy()
exog_invalid = exog_df.tail(100).copy()
exog_invalid.iloc[5, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

All series and exogenous variables are within the training range.


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_0' has one or more values outside the range seen during training [0.00022,  │
│ 0.99988]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_1' has one or more values outside the range seen during training [0.00001,  │
│ 1.00000]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

'feature_1': 'feature_1' has one or more values outside the range seen during training [0.00001, 1.00000]. 
'feature_1': 'feature_1' has one or more values outside the range seen during training [0.00001, 1.00000]. 



(True, [], ['feature_0', 'feature_1'])

In [34]:
range_detector = RangeDriftDetector()
range_detector.fit(series=y, exog=exog_df)
range_detector
exog_valid = exog_df.tail(100).copy()
exog_invalid = exog_df.tail(100).copy()
exog_invalid.iloc[5, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

All series and exogenous variables are within the training range.


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_0' has one or more values outside the range seen during training [0.00022,  │
│ 0.99988]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_1' has one or more values outside the range seen during training [0.00001,  │
│ 1.00000]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

'feature_1': 'feature_1' has one or more values outside the range seen during training [0.00001, 1.00000]. 
'feature_1': 'feature_1' has one or more values outside the range seen during training [0.00001, 1.00000]. 



(True, [], ['feature_0', 'feature_1'])

In [35]:
range_detector = RangeDriftDetector()

range_detector.fit(series=series_dict, exog=exog_dict)
print(range_detector)
exog_valid = deepcopy(exog_dict)
exog_invalid = deepcopy(exog_dict)
for key in exog_invalid.keys():
    
    exog_invalid[key].iloc[1, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)
range_detector

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(4.4901005799458105e-05), np.float64(0.9997749055410741)), 'series_2': (np.float64(6.280271749758981e-06), np.float64(0.999779891742373))} 
Exogenous value ranges: {'series_1': {'feature_0': (np.float64(0.0002191558626745138), np.float64(0.9998784961752988)), 'feature_1': (np.float64(1.3485237646770898e-05), np.float64(0.9999957682451185)), 'feature_2': (np.float64(0.0002903972354638684), np.float64(0.9999165682427562)), 'feature_3': (np.float64(9.682099526342469e-05), np.float64(0.9999938513829746)), 'feature_4': (np.float64(2.1679103396898824e-05), np.float64(0.9997456222238131)), 'cat_feature_0': {'B', 'A', 'C'}, 'cat_feature_1': {'B', 'A', 'C'}, 'cat_feature_2': {'B', 'A', 'C'}, 'cat_feature_3': {'B', 'A', 'C'}, 'cat_feature_4': {'C', 'B', 'A'}}, 'series_2': {'feature_0': (np.float64(0.0002191558626745138), np.float64(0.9998784961752988)), 'feature_1': (np.float64(1.3485237646770898e-05), np.float64(0.9999957682451185

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': 'feature_0' has one or more values outside the range seen during         │
│ training [0.00022, 0.99988]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': 'feature_1' has one or more values outside the range seen during         │
│ training [0.00001, 1.00000]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': 'feature_0' has one or more values outside the range seen during         │
│ training [0.00022, 0.99988]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': 'feature_1' has one or more values outside the range seen during         │
│ training [0.00001, 1.00000]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

'series_2': 'feature_1' has one or more values outside the range seen during training [0.00001, 1.00000]. 
'series_2': 'feature_1' has one or more values outside the range seen during training [0.00001, 1.00000]. 



RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(4.4901005799458105e-05), np.float64(0.9997749055410741)), 'series_2': (np.float64(6.280271749758981e-06), np.float64(0.999779891742373))} 
Exogenous value ranges: {'series_1': {'feature_0': (np.float64(0.0002191558626745138), np.float64(0.9998784961752988)), 'feature_1': (np.float64(1.3485237646770898e-05), np.float64(0.9999957682451185)), 'feature_2': (np.float64(0.0002903972354638684), np.float64(0.9999165682427562)), 'feature_3': (np.float64(9.682099526342469e-05), np.float64(0.9999938513829746)), 'feature_4': (np.float64(2.1679103396898824e-05), np.float64(0.9997456222238131)), 'cat_feature_0': {'B', 'A', 'C'}, 'cat_feature_1': {'B', 'A', 'C'}, 'cat_feature_2': {'B', 'A', 'C'}, 'cat_feature_3': {'B', 'A', 'C'}, 'cat_feature_4': {'C', 'B', 'A'}}, 'series_2': {'feature_0': (np.float64(0.0002191558626745138), np.float64(0.9998784961752988)), 'feature_1': (np.float64(1.3485237646770898e-05), np.float64(0.9999957682451185

In [36]:
exog_valid['series_1']['feature_1000'] = 10000